## Dataset de películas

#### Manejo del archivo original
El archivo original posee un formato incorrecto con respecto a los separadores. Primero se soluciona eso filtrando las lineas del csv que tengan menos de 9 columnas

In [155]:
import csv
import os
import collections
import random
import pandas as pd

ruta_wd = os.getcwd()
ruta_original = os.path.join(ruta_wd,"mymoviedb.csv")
ruta_arreglado = os.path.join(ruta_wd,"mymoviedb_arreglado.csv")

with open(ruta_original,encoding="utf-8") as archivo:
    csvreader = csv.reader(archivo,delimiter=',')
    lista = [linea for linea in csvreader if len(linea) == 9]

with open(ruta_arreglado,"w",encoding="utf-8") as archivo:
    writer = csv.writer(archivo,delimiter=',',lineterminator='\n')
    for linea in lista: writer.writerow(linea)

Con el archivo listo, se empieza a utilizar pandas para reordenar las columnas y descartar las innecesarias

In [156]:

archivo_original = pd.read_csv(ruta_arreglado, encoding="utf-8", delimiter=",")
columnas = archivo_original.columns

archivo_modif = archivo_original[[columnas[7],columnas[6],columnas[0],columnas[5],columnas[2],columnas[1]]]

print(archivo_modif['Original_Language'].unique())
# No hay lenguajes de mas de 2 caracteres

# Saco las overviews vacias
archivo_modif = archivo_modif[archivo_modif['Overview'] != ""]

['en' 'ja' 'fr' 'hi' 'es' 'ru' 'de' 'th' 'ko' 'tr' 'cn' 'zh' 'it' 'pt'
 'ml' 'pl' 'fi' 'no' 'da' 'id' 'sv' 'nl' 'te' 'sr' 'is' 'ro' 'tl' 'fa'
 'uk' 'nb' 'eu' 'lv' 'ar' 'el' 'cs' 'ms' 'bn' 'ca' 'la' 'ta' 'hu' 'he'
 'et']


#### Contador de palabras de todas las overviews

In [157]:
contador = collections.Counter()

overviews = archivo_modif["Overview"]
overviews = overviews.apply(lambda x: x.lower().replace(",","").replace(".","").replace("-",""))

overviews_palabras = overviews.str.split()

contador = collections.Counter()
overviews_palabras.apply(lambda x: contador.update(x))

palabras_mas_comunes = list(map(lambda x: x[0],contador.most_common(100)))
print(palabras_mas_comunes)

['the', 'a', 'to', 'and', 'of', 'in', 'his', 'is', 'with', 'her', 'an', 'he', 'for', 'on', 'that', 'their', 'when', 'as', 'by', 'who', 'from', 'but', 'they', 'she', 'are', 'has', 'after', 'into', 'at', 'him', 'life', 'new', 'one', 'young', 'it', 'up', 'out', 'world', 'be', 'must', 'two', 'this', 'will', 'have', 'all', 'family', 'them', 'find', 'man', 'only', 'love', 'while', 'where', 'friends', 'finds', 'years', 'time', 'story', 'about', 'not', 'woman', 'help', 'school', 'get', 'home', 'back', 'was', 'can', 'mysterious', 'own', 'what', 'against', 'now', 'take', 'group', 'which', 'save', 'way', 'before', 'more', 'first', 'soon', 'girl', 'other', 'been', 'himself', 'through', 'war', 'father', 'become', 'over', 'lives', 'becomes', 'most', 'town', 'being', 'takes', 'down', 'during', 'team']


#### Funciones para transformar las sinopsis

In [158]:
def eliminar_palabras_comunes(overview,palabras_comunes):
    """ Recibe una sinopsis de pelicula en string, le elimina las palabras mas comunes, y la retorna en tipo lista """
    overview_filtrado = list(filter(lambda x: x not in palabras_comunes,overview))
    return overview_filtrado

def reducir_overview_random(overview):
    """ Recibe una sinopsis en lista, la desordena, y retorna 3 palabras al azar, separadas con ";" en una string """
    if len(overview) < 3: 
        random.shuffle(overview)
        overview_reducida = overview
    else :
        random.shuffle(overview)
        overview_reducida = random.sample(overview,3)
    overview_reducida = [elem.strip().strip('"') for elem in overview_reducida]
    overview_reducida_str = "; ".join(overview_reducida)
    return overview_reducida_str

def modificar_overview(overview,palabras_comunes):
    """ Recibe una lista de palabras (la overview) y devuelve un string en formato solicitado en base a esa lista """
    ov = eliminar_palabras_comunes(overview,palabras_comunes)
    return reducir_overview_random(ov)

overviews_modif = overviews_palabras.apply(modificar_overview,args=(palabras_mas_comunes,))

#### Ejemplo de cómo queda el overview de una película

In [159]:
print("-"*50)
print(f"Sinopsis original \n\n {archivo_modif.iloc[0,4]}\n")
overview_sin_comunes = eliminar_palabras_comunes(overviews_palabras.iloc[0],palabras_mas_comunes)
print("-"*50)
print(f"Sinopsis sin las 100 palabras más comunes\n\n{overview_sin_comunes}\n")
print("-"*50)
overview_final = reducir_overview_random(overview_sin_comunes)
print(f"Sinopsis en 3 palabras al azar\n\n{overview_final}")

--------------------------------------------------
Sinopsis original 

 Peter Parker is unmasked and no longer able to separate his normal life from the high-stakes of being a super-hero. When he asks for help from Doctor Strange the stakes become even more dangerous, forcing him to discover what it truly means to be Spider-Man.

--------------------------------------------------
Sinopsis sin las 100 palabras más comunes

['peter', 'parker', 'unmasked', 'no', 'longer', 'able', 'separate', 'normal', 'highstakes', 'superhero', 'asks', 'doctor', 'strange', 'stakes', 'even', 'dangerous', 'forcing', 'discover', 'truly', 'means', 'spiderman']

--------------------------------------------------
Sinopsis en 3 palabras al azar

discover; truly; superhero


#### Generación del nuevo dataset
Se reemplazan las overviews por las nuevas generadas y se guarda el archivo

In [160]:
archivo_modif["Overview"] = overviews_modif

ruta_nuevo = os.path.join(ruta_wd,"peliculas_formateado.csv")
archivo_modif.to_csv(ruta_nuevo, index=False)